In [1]:
import requests

def fetch_stock_footage_pexels(prompt):
    url = f"https://api.pexels.com/videos/search?query={prompt}&size=large&orientation=portrait&per_page=5"
    headers = {"Authorization": "WfRAG5IUtSN3kSHJCXJswuGA6MfqQbRnc6ZvxQqi4Z1vXjXSgdCydi2j"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return [video['video_files'][0]['link'] for video in response.json().get('videos', [])], response.json()
    return []


links = fetch_stock_footage_pexels("gym clothing advertisements")

In [4]:
import json

In [19]:
time_cap=json.loads(open("sample_timed_scripts.json", "r").read())

In [20]:
type(time_cap), time_cap

(list,
 [{'start_time': 0.0, 'end_time': 0.7, 'caption': 'Calling all'},
  {'start_time': 0.7, 'end_time': 1.4, 'caption': 'Sari lovers'},
  {'start_time': 1.4, 'end_time': 3.08, 'caption': 'Diwali is'},
  {'start_time': 3.08, 'end_time': 3.64, 'caption': 'here and'},
  {'start_time': 3.64, 'end_time': 3.88, 'caption': 'we are'},
  {'start_time': 3.88, 'end_time': 4.34, 'caption': 'celebrating'},
  {'start_time': 4.34, 'end_time': 5.1, 'caption': 'with a dazzling'},
  {'start_time': 5.1, 'end_time': 6.58, 'caption': 'sail Get'},
  {'start_time': 6.58, 'end_time': 7.04, 'caption': 'ready to'},
  {'start_time': 7.04, 'end_time': 7.36, 'caption': 'light up'},
  {'start_time': 7.36, 'end_time': 7.88, 'caption': 'your wardrobe'},
  {'start_time': 7.88, 'end_time': 8.3, 'caption': 'with our'},
  {'start_time': 8.3, 'end_time': 8.76, 'caption': 'exquisite'},
  {'start_time': 8.76, 'end_time': 9.42, 'caption': 'collection of'},
  {'start_time': 9.42, 'end_time': 10.2, 'caption': 'serres now'},

In [21]:
objects_=[]
for ind, i in enumerate(time_cap):
    print(ind, i, end="\r")
    if i.get("caption", None):
        links = fetch_stock_footage_pexels(i.get("caption", None))
        objects_.append({"caption":i.get("caption"), "links":links})
        
with open("objects_from_pexels.json", "w") as f:
    json.dump(objects_, f)

In [2]:
import requests
from urllib.parse import quote_plus

def fetch_stock_footage(prompt):
    encoded_prompt = quote_plus(prompt)  # Encodes spaces as +
    print(encoded_prompt)
    url = f"https://pixabay.com/api/videos/?key=47559207-6a059c5662f5f2564ce2379f2&q={encoded_prompt}&orientation=vertical&per_page=5"
    
    headers = {
        "Content-Type": "application/json",  # Set the Content-Type header
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
       
        return [video['videos']["small"]["url"] for video in response.json().get('hits', [])]
    else:
        print(f"Error: {response.status_code}")
        print(response.text)  # Print the error message from the response
    return []

links = fetch_stock_footage("gym clothing advertisement")
print(links)


gym+clothing+advertisement
['https://cdn.pixabay.com/video/2015/10/16/1006-142621176_small.mp4', 'https://cdn.pixabay.com/video/2020/06/27/43238-435970498_small.mp4', 'https://cdn.pixabay.com/video/2024/03/29/206029_small.mp4', 'https://cdn.pixabay.com/video/2023/01/27/148208-793717949_small.mp4', 'https://cdn.pixabay.com/video/2021/06/16/77916-563974349_small.mp4']


In [3]:
links

['https://cdn.pixabay.com/video/2015/10/16/1006-142621176_small.mp4',
 'https://cdn.pixabay.com/video/2020/06/27/43238-435970498_small.mp4',
 'https://cdn.pixabay.com/video/2024/03/29/206029_small.mp4',
 'https://cdn.pixabay.com/video/2023/01/27/148208-793717949_small.mp4',
 'https://cdn.pixabay.com/video/2021/06/16/77916-563974349_small.mp4']

In [8]:
from moviepy import VideoFileClip, concatenate_videoclips, TextClip, CompositeVideoClip
def create_video_with_captions(video_links, script_text, output_file):
    try:
        clips = []
        captions = script_text.split('.')
        
        for idx, video_link in enumerate(video_links):
            clip = VideoFileClip(video_link).subclipped(0, 5)  # Use only first 5 seconds
            
            if idx < len(captions):
                caption_text = captions[idx].strip()
                if caption_text:  # Ensure there's valid text
                    caption = (
                        TextClip(
                            caption_text,
                            fontsize=50,
                            color='white',
                            font="Arial",
                            bg_color='black',
                        )
                        .set_duration(clip.duration)
                        .set_position(('center', 'bottom'))  # Adjust position
                    )
                    clip = CompositeVideoClip([clip, caption])
            
            clips.append(clip)
        
        final_video = concatenate_videoclips(clips, method="compose")
        final_video.write_videofile(output_file, codec="libx264", fps=24)
        return output_file

    except Exception as e:
        print(e)
        return None

# def create_video_with_captions(video_links, script_text, output_file):
#     clips = []
#     captions = script_text.split('.')
#     for idx, video_link in enumerate(video_links):
#         clip = VideoFileClip(video_link).subclipped(0, 5)  # Use only first 5 seconds

#         if idx < len(captions):
#             caption = TextClip(text=captions[idx], font="calibri",font_size=72, color='white', bg_color='black',duration=clip.duration,horizontal_align="center",vertical_align="center")
#             # caption = caption.set_duration(clip.duration).set_position(('center', 'bottom'))
#             clip = CompositeVideoClip([clip, caption])
#         clips.append(clip)

#     final_video = concatenate_videoclips(clips, method="compose")
#     final_video.write_videofile(output_file, codec="libx264", fps=24)

script = """"
Elevate your workout wardrobe with our premium gym clothing collection! Designed for comfort, style, and performance, our apparel helps you push your limits while looking great. From moisture-wicking t-shirts and breathable leggings to supportive sports bras and versatile shorts, we have everything you need to stay active and confident. Made with high-quality fabrics that move with you, our gym wear is perfect for every workout, from intense HIIT sessions to relaxing yoga flows. With sleek designs and a perfect fit, you’ll feel empowered to tackle any challenge. Shop now and upgrade your fitness gear for the ultimate workout experience!"""

In [9]:
create_video_with_captions(links,script,"output6.mp4")

multiple values for argument 'font'


In [7]:
def add_background_music(video_file, music_file, output_file):
    video = VideoFileClip(video_file)
    music = VideoFileClip(music_file).audio
    video = video.set_audio(music)
    video.write_videofile(output_file, codec="libx264", fps=24)
